In [1]:
from util import make_document

DIR_PATH = "chinese_data/"

documents = make_document(DIR_PATH)
len(documents)

45

In [2]:
import jieba
from rank_bm25 import BM25Okapi

# 使用 jieba 對每篇文檔進行分詞
tokenized_documents = [list(jieba.cut(doc)) for doc in documents]

# 建立 BM25 模型
bm25 = BM25Okapi(tokenized_documents)


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\weiso131\AppData\Local\Temp\jieba.cache
Loading model cost 0.453 seconds.
Prefix dict has been built successfully.


In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 加載模型和tokenizer
model_name = "fnlp/bart-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

C:\Users\weiso131\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
import jieba.analyse
query = "記憶源點的功能為何"

# 使用 jieba 提取關鍵詞（例如提取前5個關鍵詞）
keywords = jieba.analyse.extract_tags(query, topK=5)

print("提取出的關鍵詞:", keywords)

keywords_query = ""
for k in keywords:
    keywords_query += k + " "
keywords_query

提取出的關鍵詞: ['記憶源點', '為何', '功能']


'記憶源點 為何 功能 '

In [22]:

# 使用 jieba 將查詢進行分詞
tokenized_query = list(jieba.cut(keywords_query))

# 計算每篇文檔的分數
doc_scores = bm25.get_scores(tokenized_query)

top_n = bm25.get_top_n(tokenized_query, documents, n=3)
retrieved_answers = ""
# 列出前 3 篇最相關的文檔
for idx, doc in enumerate(top_n):
    print(f"Top {idx + 1} 相關文檔: {doc}")
    retrieved_answers += doc

retrieved_answers


Top 1 相關文檔: 官方曾特別提醒玩家切勿購買第三方商家出售的記憶源點。  

Top 2 相關文檔: 在遊戲內可用於解鎖已購買 /免費歌曲譜面。在移動版中也可用於購買世界模式
體力，擴充好友數上限，或在 Legacy模式地圖 /Lost Chapter:Beyond/Breached 
Chapter 章節地圖獲得增益。在 Nintendo Switch 版中還可以用於升級搭檔。  
 記憶源點  
遊戲內付費道具，可以於官方網站購買，或 iOS版使用App Store 內購購買。
部分情況下官方也可能會免費發放記憶源點。  

Top 3 相關文檔: 在Lost Chapter:Beyond 和Breach Chapter 中，完全捨棄了其餘章節的階梯形
式地圖，改用以百分數呈現的「 Beyond計量值」 。對於 Lost章節，該獎勵與遊
玩結果，該地圖對選擇搭檔加成的「相性契合」值及搭檔的 OVER值有關；對
於Breach章節，該獎勵與遊玩結果，每個地圖獨有的「鐫刻法則」有關，有時
也和搭檔的 OVER值有關。玩家也可通過使用殘片，記憶源點和 /或Beyond強
化計量條來提升遊玩加成。  



'官方曾特別提醒玩家切勿購買第三方商家出售的記憶源點。  \n在遊戲內可用於解鎖已購買 /免費歌曲譜面。在移動版中也可用於購買世界模式\n體力，擴充好友數上限，或在 Legacy模式地圖 /Lost Chapter:Beyond/Breached \nChapter 章節地圖獲得增益。在 Nintendo Switch 版中還可以用於升級搭檔。  \n 記憶源點  \n遊戲內付費道具，可以於官方網站購買，或 iOS版使用App Store 內購購買。\n部分情況下官方也可能會免費發放記憶源點。  \n在Lost Chapter:Beyond 和Breach Chapter 中，完全捨棄了其餘章節的階梯形\n式地圖，改用以百分數呈現的「 Beyond計量值」 。對於 Lost章節，該獎勵與遊\n玩結果，該地圖對選擇搭檔加成的「相性契合」值及搭檔的 OVER值有關；對\n於Breach章節，該獎勵與遊玩結果，每個地圖獨有的「鐫刻法則」有關，有時\n也和搭檔的 OVER值有關。玩家也可通過使用殘片，記憶源點和 /或Beyond強\n化計量條來提升遊玩加成。  \n'

In [23]:


# 示例文本
input_text = f"前後文:{retrieved_answers}\n\n 提問:{query}\n\n 答案:"

# 將文本轉換為模型可讀的格式
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# 移除 'token_type_ids' (BART 不需要這個)
inputs.pop('token_type_ids', None)
# 生成結果
outputs = model.generate(**inputs, max_length=50, num_beams=5, early_stopping=True)

# 解碼生成的文本
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("生成的文本:", generated_text)

生成的文本: 前 後 文 : 官 方 曾 特 別 提 醒 玩 家 切 勿 購 買 第 三 方 商 家 出 售 的 記 憶 源 點 可 以 在 遊 戲 內 可 用 於 解 鎖 已 購 買 / 免 費 歌
